# MEDIA BILLING RECONCILIATION WITH AGENTIC AI​

---

## DESCRIPTION

This notebook implements a **File Reader Agent** that can ingest media invoices and billing data in multiple formats and extract structured information for processing.

The agent uses CrewAI to intelligently read and parse files, making it easy to integrate with billing reconciliation workflows.

---

## 🎯 CAPABILITIES

**File Format Support:**
- 📄 **PDF** - Extract text from invoice PDFs
- 📊 **CSV** - Parse tabular billing data
- 📈 **Excel** (.xlsx, .xls) - Read spreadsheet data
- 🖼️ **Images** - OCR text extraction from scanned documents
- 📝 **Text** - Plain text file reading
- 🔖 **XML** - Parse XML structured data

**Agent Features:**
- Single unified agent for all file types
- Automatic file type detection
- Structured data extraction
- Error handling and validation
- Ready for CrewAI integration

---

## 🤖 WHAT WE'RE BUILDING

A **File Reader Agent** that:
1. Accepts file paths as input
2. Automatically detects file format
3. Extracts and structures the data
4. Returns clean, parsed content
5. Integrates with CrewAI workflows

---

## 📦 USE CASE

This agent serves as the **data ingestion layer** for the media billing reconciliation system:
- **Input**: Invoice files (PDF, CSV, Excel, images, XML)
- **Process**: Read, parse, and structure data
- **Output**: Standardized data for reconciliation agents

## ✨ Enhanced Excel Reading Capabilities

The Excel file reader has been significantly improved with the following features:

### 🎯 What's New:

1. **Multi-Sheet Support**
   - Automatically detects all sheets in Excel files
   - Reads all sheets or specific sheets by name/index
   - Provides sheet-by-sheet analysis

2. **Intelligent Data Analysis**
   - Column data type detection (numeric, text, dates)
   - Statistical summaries for numeric columns
   - Missing value analysis with percentages
   - Unique value counting for pattern detection

3. **Invoice/Billing Detection**
   - Automatically identifies invoice-related data
   - Detects columns containing: invoice numbers, dates, amounts, vendors, etc.
   - Flags sheets that likely contain billing information

4. **Better Readability**
   - Structured output with clear sections
   - Metadata about file and sheet structure
   - Preview of data with proper formatting
   - Row and column summaries

5. **Helper Functions**
   - `analyze_excel_structure()` - Deep structural analysis
   - `get_excel_summary()` - Quick overview with concise mode
   - `get_excel_column_summary()` - Just column names (very lightweight)
   - `read_excel_content()` - Full content extraction with metadata

### ⚡ Optimization for Large Files

**NEW**: The reader is now optimized to prevent token overflow errors:
- Limits to **50 rows per sheet** by default (configurable)
- Shows **first 3 sheets only** when reading all sheets
- **Verbose mode disabled** by default (no detailed statistics)
- **Auto-truncates** output if > 25,000 characters
- Column width limited to 30 characters in preview

**For large Excel files:**
```python
# Option 1: Read specific sheet only
content = read_excel_content('large_file.xlsx', sheet_name='Sheet1')

# Option 2: Get just column structure (no data)
columns = get_excel_column_summary('large_file.xlsx')

# Option 3: Use concise summary
summary = get_excel_summary('large_file.xlsx', concise=True)
```

---


In [114]:
# ============================================
# INSTALL REQUIRED PACKAGES
# ============================================
# Uncomment and run to install all dependencies

# !pip install pdfplumber pandas openpyxl pillow pytesseract lxml
# !pip install crewai python-dotenv

## 📦 Step 1: Import Required Libraries

Import all necessary libraries for file reading and CrewAI integration.

In [115]:
# ============================================
# IMPORT ALL REQUIRED LIBRARIES
# ============================================
import os
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv

# File reading libraries
import pdfplumber
from PIL import Image
import pytesseract
import xml.etree.ElementTree as ET

# CrewAI imports
from crewai import Agent, Task, Crew, Process, LLM
from crewai.tools import tool

print("✅ All libraries imported successfully!")
print("   • File readers: pdfplumber, pandas, PIL, pytesseract, xml")
print("   • CrewAI: Agent, Task, Crew, LLM")

✅ All libraries imported successfully!
   • File readers: pdfplumber, pandas, PIL, pytesseract, xml
   • CrewAI: Agent, Task, Crew, LLM


In [140]:
# ============================================
# FILE READER FUNCTIONS
# ============================================

def read_pdf_content(pdf_path: str) -> str:
    """Read and extract text from PDF files."""
    try:
        full_text = ""
        with pdfplumber.open(pdf_path) as pdf:
            for page_num, page in enumerate(pdf.pages, 1):
                text = page.extract_text()
                if text:
                    full_text += f"\n--- Page {page_num} ---\n{text}\n"
        return full_text.strip()
    except Exception as e:
        return f"Error reading PDF: {str(e)}"


def read_csv_content(csv_path: str) -> str:
    """Read and parse CSV files."""
    try:
        df = pd.read_csv(csv_path)
        return df.to_string()
    except Exception as e:
        return f"Error reading CSV: {str(e)}"


def read_excel_content(excel_path: str, sheet_name=None, max_rows=50, verbose=False) -> str:
    """
    Read and parse Excel files with enhanced capabilities.
    
    Features:
    - Auto-detects and lists all sheets
    - Reads multiple sheets or a specific sheet
    - Provides data summary and structure
    - Optimized for LLM consumption (prevents token overflow)
    
    Args:
        excel_path: Path to the Excel file
        sheet_name: Specific sheet name/index to read, or None for all sheets
        max_rows: Maximum rows to display per sheet (default: 50, reduced for LLM)
        verbose: If True, include detailed statistics (use False for large files)
    
    Returns:
        Formatted string with Excel content and metadata
    """
    try:
        # Load Excel file to get sheet information
        excel_file = pd.ExcelFile(excel_path)
        sheet_names = excel_file.sheet_names
        
        output = []
        output.append(f"📊 EXCEL FILE: {Path(excel_path).name}")
        output.append(f"Total Sheets: {len(sheet_names)} - {', '.join(sheet_names)}")
        output.append(f"{'='*70}\n")
        
        # Determine which sheets to read
        sheets_to_read = []
        if sheet_name is not None:
            sheets_to_read = [sheet_name]
        else:
            # Read all sheets (limit to first 3 for large files to prevent token overflow)
            sheets_to_read = sheet_names[:3]
            if len(sheet_names) > 3:
                output.append(f"⚠️  Showing first 3 of {len(sheet_names)} sheets (to avoid data overflow)\n")
        
        # Read and analyze each sheet
        for idx, sheet in enumerate(sheets_to_read, 1):
            try:
                df = pd.read_excel(excel_path, sheet_name=sheet)
                
                output.append(f"\n📄 SHEET {idx}: '{sheet}'")
                output.append(f"{'-'*70}")
                output.append(f"Size: {len(df)} rows × {len(df.columns)} columns")
                
                # Column info - summarized
                output.append(f"\nColumns ({len(df.columns)}):")
                for col in df.columns:
                    dtype = df[col].dtype
                    non_null = df[col].notna().sum()
                    unique = df[col].nunique()
                    output.append(f"  • {col}: {dtype} ({non_null} non-null, {unique} unique)")
                
                # Determine how many rows to show
                rows_to_show = min(max_rows, len(df))
                
                # Data preview - limited rows
                output.append(f"\nData Preview (showing {rows_to_show} of {len(df)} rows):")
                output.append(df.head(rows_to_show).to_string(index=True, max_colwidth=30))
                
                # Only include detailed stats if verbose=True
                if verbose:
                    # Statistical summary for numeric columns
                    numeric_cols = df.select_dtypes(include=['number']).columns
                    if len(numeric_cols) > 0:
                        output.append(f"\nNumeric Summary:")
                        output.append(df[numeric_cols].describe().to_string())
                    
                    # Missing data
                    missing_data = df.isnull().sum()
                    if missing_data.any():
                        output.append(f"\nMissing Values:")
                        for col, count in missing_data[missing_data > 0].items():
                            pct = (count / len(df)) * 100
                            output.append(f"  • {col}: {count} ({pct:.1f}%)")
                else:
                    # Just show totals for missing data
                    total_missing = df.isnull().sum().sum()
                    if total_missing > 0:
                        output.append(f"\nTotal missing values: {total_missing}")
                
                output.append(f"\n{'-'*70}")
                
            except Exception as sheet_error:
                output.append(f"\n⚠️  Error reading sheet '{sheet}': {str(sheet_error)}\n")
        
        result = "\n".join(output)
        
        # Warn if output is very large
        if len(result) > 25000:
            return f"⚠️ Excel file is very large ({len(result)} chars). Consider using sheet_name parameter.\n\n" + result[:25000] + f"\n\n... [Output truncated, use verbose=False or specify sheet_name]"
        
        return result
        
    except Exception as e:
        return f"Error reading Excel: {str(e)}"


def read_image_content(image_path: str) -> str:
    """Extract text from images using OCR."""
    try:
        image = Image.open(image_path)
        text = pytesseract.image_to_string(image)
        return text.strip()
    except Exception as e:
        return f"Error reading image: {str(e)}"


def read_xml_content(xml_path: str) -> str:
    """Parse and read XML files."""
    try:
        tree = ET.parse(xml_path)
        root = tree.getroot()
        return ET.tostring(root, encoding='unicode')
    except Exception as e:
        return f"Error reading XML: {str(e)}"


def read_text_content(text_path: str) -> str:
    """Read plain text files."""
    try:
        with open(text_path, 'r', encoding='utf-8') as f:
            return f.read()
    except Exception as e:
        return f"Error reading text file: {str(e)}"

print("✅ Individual file reader functions created!")
print("   • read_pdf_content()")
print("   • read_csv_content()")
print("   • read_excel_content() - OPTIMIZED for large files!")
print("   • read_image_content()")
print("   • read_xml_content()")
print("   • read_text_content()")


✅ Individual file reader functions created!
   • read_pdf_content()
   • read_csv_content()
   • read_excel_content() - OPTIMIZED for large files!
   • read_image_content()
   • read_xml_content()
   • read_text_content()


In [148]:
# ============================================
# EXCEL-SPECIFIC HELPER FUNCTIONS
# ============================================

def analyze_excel_structure(excel_path: str) -> dict:
    """
    Deep analysis of Excel file structure.
    Returns detailed information about sheets, data patterns, and potential invoice data.
    
    Returns:
        Dictionary with analysis results including:
        - Sheet information
        - Detected tables and data regions
        - Potential header rows
        - Column relationships
    """
    try:
        excel_file = pd.ExcelFile(excel_path)
        analysis = {
            'file_name': Path(excel_path).name,
            'total_sheets': len(excel_file.sheet_names),
            'sheet_names': excel_file.sheet_names,
            'sheets_analysis': {}
        }
        
        for sheet_name in excel_file.sheet_names:
            df = pd.read_excel(excel_path, sheet_name=sheet_name)
            
            # Analyze this sheet
            sheet_info = {
                'rows': len(df),
                'columns': len(df.columns),
                'column_names': list(df.columns),
                'has_unnamed_columns': any('Unnamed' in str(col) for col in df.columns),
                'numeric_columns': list(df.select_dtypes(include=['number']).columns),
                'text_columns': list(df.select_dtypes(include=['object']).columns),
                'date_columns': list(df.select_dtypes(include=['datetime']).columns),
                'empty_rows': df.isnull().all(axis=1).sum(),
                'duplicate_rows': df.duplicated().sum(),
            }
            
            # Try to detect if this looks like invoice/billing data
            invoice_indicators = []
            keywords = ['invoice', 'bill', 'amount', 'total', 'price', 'quantity', 'date', 'vendor', 'payment']
            
            for col in df.columns:
                col_str = str(col).lower()
                for keyword in keywords:
                    if keyword in col_str:
                        invoice_indicators.append(f"Column '{col}' contains '{keyword}'")
            
            # Check for data in cells
            for col in df.columns:
                sample_values = df[col].dropna().astype(str).head(5).tolist()
                for keyword in keywords:
                    if any(keyword in str(val).lower() for val in sample_values):
                        invoice_indicators.append(f"Column '{col}' has values matching '{keyword}'")
                        break
            
            sheet_info['invoice_indicators'] = invoice_indicators
            sheet_info['likely_invoice_data'] = len(invoice_indicators) > 0
            
            analysis['sheets_analysis'][sheet_name] = sheet_info
        
        return analysis
        
    except Exception as e:
        return {'error': str(e)}


def get_excel_summary(excel_path: str, concise=True) -> str:
    """
    Get a summary of Excel file suitable for LLM understanding.
    
    Args:
        excel_path: Path to Excel file
        concise: If True, returns brief summary. If False, more detailed.
    """
    analysis = analyze_excel_structure(excel_path)
    
    if 'error' in analysis:
        return f"Error analyzing Excel file: {analysis['error']}"
    
    summary = []
    summary.append(f"📊 Excel File: {analysis['file_name']}")
    summary.append(f"Total Sheets: {analysis['total_sheets']}")
    
    if concise:
        # Brief summary
        summary.append(f"Sheets: {', '.join(analysis['sheet_names'])}")
        total_rows = sum(info['rows'] for info in analysis['sheets_analysis'].values())
        summary.append(f"Total Data Rows: {total_rows}")
    else:
        # Detailed summary
        summary.append("")
        for sheet_name, info in analysis['sheets_analysis'].items():
            summary.append(f"Sheet: {sheet_name}")
            summary.append(f"  Size: {info['rows']} rows × {info['columns']} columns")
            summary.append(f"  Columns: {', '.join([str(c) for c in info['column_names'][:10]])}")
            
            if info['likely_invoice_data']:
                summary.append(f"  ✅ Likely contains billing/invoice data")
                summary.append(f"  Indicators: {'; '.join(info['invoice_indicators'][:3])}")
            
            summary.append("")
    
    return "\n".join(summary)


def get_excel_column_summary(excel_path: str, sheet_name=None) -> str:
    """
    Get just the column names and types from an Excel file.
    Useful for understanding structure without loading all data.
    """
    try:
        excel_file = pd.ExcelFile(excel_path)
        sheets = [sheet_name] if sheet_name else excel_file.sheet_names[:3]
        
        summary = []
        summary.append(f"📋 Column Structure: {Path(excel_path).name}")
        
        for sheet in sheets:
            df = pd.read_excel(excel_path, sheet_name=sheet, nrows=0)  # Just headers
            summary.append(f"\nSheet: {sheet}")
            summary.append(f"Columns: {', '.join([str(c) for c in df.columns])}")
        
        return "\n".join(summary)
    except Exception as e:
        return f"Error: {str(e)}"


print("✅ Excel analysis helper functions created!")
print("   • analyze_excel_structure() - Deep Excel analysis")
print("   • get_excel_summary() - Quick summary (with concise mode)")
print("   • get_excel_column_summary() - Just column names (very lightweight)")


✅ Excel analysis helper functions created!
   • analyze_excel_structure() - Deep Excel analysis
   • get_excel_summary() - Quick summary (with concise mode)
   • get_excel_column_summary() - Just column names (very lightweight)


In [142]:
# ============================================
# UNIVERSAL FILE READER TOOL (CrewAI Tool)
# ============================================

@tool("read_file_tool")
def read_file_tool(file_path: str, sheet_name: str = None) -> str:
    """
    Universal file reader tool that automatically detects and reads various file formats.
    Supports: PDF, CSV, Excel (with multi-sheet support), Images (OCR), XML, and Text files.
    
    OPTIMIZED for AI agents - prevents token overflow on large Excel files.
    
    Args:
        file_path: Path to the file to read
        sheet_name: (Optional) For Excel files, specify sheet name or index to read.
                   If None, reads first 3 sheets. Examples: "Sheet1", 0, "Invoice Data"
        
    Returns:
        Extracted content as string with detailed formatting and metadata
    """
    file_path = Path(file_path)
    
    if not file_path.exists():
        return f"❌ Error: File not found at {file_path}"
    
    suffix = file_path.suffix.lower()
    
    print(f"📂 Reading file: {file_path.name} (type: {suffix})")
    
    # Route to appropriate reader based on file extension
    if suffix == '.pdf':
        content = read_pdf_content(str(file_path))
    elif suffix == '.csv':
        content = read_csv_content(str(file_path))
    elif suffix in ['.xlsx', '.xls']:
        # Use enhanced Excel reader with optimization for LLM
        print(f"   Using optimized Excel reader (max 50 rows/sheet)...")
        if sheet_name:
            print(f"   Reading specific sheet: {sheet_name}")
        # Use verbose=False to prevent token overflow
        content = read_excel_content(str(file_path), sheet_name=sheet_name, max_rows=50, verbose=False)
    elif suffix in ['.png', '.jpg', '.jpeg', '.bmp', '.tiff']:
        content = read_image_content(str(file_path))
    elif suffix == '.xml':
        content = read_xml_content(str(file_path))
    elif suffix == '.txt':
        content = read_text_content(str(file_path))
    else:
        content = f"⚠️ Unsupported file type: {suffix}"
    
    print(f"✅ Successfully read {len(content)} characters")
    return content

print("✅ CrewAI File Reader Tool created!")
print("   Tool name: 'read_file_tool'")
print("   Supports: PDF, CSV, Excel (Optimized!), Images, XML, Text")
print("   Excel Features: Auto-limited to 50 rows/sheet to prevent token overflow")


✅ CrewAI File Reader Tool created!
   Tool name: 'read_file_tool'
   Supports: PDF, CSV, Excel (Optimized!), Images, XML, Text
   Excel Features: Auto-limited to 50 rows/sheet to prevent token overflow


In [144]:
# ============================================
# ADDITIONAL HELPER TOOL FOR EXCEL METADATA
# ============================================

@tool("get_excel_info")
def get_excel_info(file_path: str) -> str:
    """
    Get lightweight metadata about an Excel file without loading all data.
    Use this FIRST before reading full Excel content to understand structure.
    
    Returns:
    - File name
    - Total sheets and their names
    - Total rows across all sheets
    - Brief indication if likely contains invoice/billing data
    
    This is very fast and uses minimal tokens.
    """
    file_path = Path(file_path)
    
    if not file_path.exists():
        return f"❌ Error: File not found at {file_path}"
    
    if file_path.suffix.lower() not in ['.xlsx', '.xls']:
        return f"⚠️ Not an Excel file: {file_path.suffix}"
    
    return get_excel_summary(str(file_path), concise=True)

print("✅ Excel metadata tool created!")
print("   Tool name: 'get_excel_info'")
print("   Purpose: Lightweight Excel file inspection (minimal tokens)")


✅ Excel metadata tool created!
   Tool name: 'get_excel_info'
   Purpose: Lightweight Excel file inspection (minimal tokens)


In [118]:
# ============================================
# LOAD ENVIRONMENT AND INITIALIZE LLM
# ============================================

load_dotenv()

# Get OpenAI API key from environment
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if not OPENAI_API_KEY:
    print("⚠️  Warning: OPENAI_API_KEY not found in environment")
    print("   Please add it to your .env file")
else:
    print("✅ OpenAI API key loaded")

# Initialize LLM
llm = LLM(
    model="gpt-4o",
    api_key=OPENAI_API_KEY,
    temperature=0.7
)

print("✅ LLM initialized: gpt-4o")

✅ OpenAI API key loaded
✅ LLM initialized: gpt-4o


In [145]:
# ============================================
# CREATE FILE READER AGENT
# ============================================

file_reader_agent = Agent(
    role='File Data Extraction Specialist',
    goal='Read and extract data from various file formats including PDF, CSV, Excel, images, XML, and text files',
    backstory="""You are an expert in data extraction and file parsing. 
    You can read any type of document - from PDFs and spreadsheets to scanned images 
    and XML files. You extract information accurately and structure it in a clear, 
    usable format. You handle invoices, billing documents, and financial records 
    with precision.
    
    IMPORTANT: For Excel files, always use get_excel_info FIRST to understand 
    the file structure before reading full content. This prevents token overflow.""",
    llm=llm,
    tools=[get_excel_info, read_file_tool],
    verbose=True,
    allow_delegation=False
)

print("✅ File Reader Agent created!")
print(f"   Role: {file_reader_agent.role}")
print(f"   Tools: {[tool.name for tool in file_reader_agent.tools]}")
print("   ⚡ Optimized for large Excel files!")


✅ File Reader Agent created!
   Role: File Data Extraction Specialist
   Tools: ['get_excel_info', 'read_file_tool']
   ⚡ Optimized for large Excel files!


In [120]:
# ============================================
# CREATE FILE READING TASK
# ============================================

def create_file_reading_task(file_path: str, extraction_focus: str = "all data") -> Task:
    """
    Create a task for reading and extracting data from a file.
    
    Args:
        file_path: Path to the file to read
        extraction_focus: What specific data to focus on (default: "all data")
    
    Returns:
        Task object configured for file reading
    """
    task = Task(
        description=f"""Read and extract data from the following file:
        
File Path: {file_path}
Extraction Focus: {extraction_focus}

Your tasks:
1. Use the read_file_tool to read the file
2. Extract and structure the relevant information
3. If it's an invoice or billing document, identify:
   - Vendor/supplier name
   - Invoice number and date
   - Line items with descriptions and amounts
   - Total amount
   - Any other relevant billing information
4. Present the extracted data in a clear, structured format

Provide a comprehensive summary of the file contents.""",
        agent=file_reader_agent,
        expected_output="""A structured summary of the file contents including:
        - File type and basic metadata
        - Key data points extracted
        - For invoices: vendor, invoice number, date, line items, total
        - Any notable information or patterns found"""
    )
    
    return task

print("✅ File reading task creator function defined!")
print("   Usage: task = create_file_reading_task('path/to/file.pdf')")

✅ File reading task creator function defined!
   Usage: task = create_file_reading_task('path/to/file.pdf')


## 📋 Define Canonical Invoice Schema

This schema defines the standard structure for extracted invoice data with clear validation rules.


In [154]:
# ============================================
# CANONICAL INVOICE SCHEMA & EXTRACTION RULES
# ============================================

CANONICAL_SCHEMA_DOC = """
You MUST output a JSON object matching this EXACT structure:

{
  "invoice_header": {
    "invoice_number": string or null,
    "vendor_name": string or null,
    "campaign_name": string or null,
    "invoice_date": string or null,          // YYYY-MM-DD if possible
    "billing_start_date": string or null,    // YYYY-MM-DD if possible
    "billing_end_date": string or null,      // YYYY-MM-DD if possible
    "currency": string or null,
    "total_impressions": number or null,
    "total_views": number or null,           // complete views, video views, conversions, etc.
    "total_clicks": number or null,
    "gross_revenue": number or null,
    "net_revenue": number or null,
    "total_discount_amount": number or null,
    "discount_percent": number or null,
    "profit": number or null
  },
  "line_items": [
    {
      "line_id": integer,
      "campaign_name": string or null,
      "placement": string or null,
      "start_date": string or null,          // YYYY-MM-DD if possible
      "end_date": string or null,            // YYYY-MM-DD if possible
      "planned_impressions": number or null,
      "billed_impressions": number or null,
      "views": number or null,               // complete views, video views, conversions, etc.
      "clicks": number or null,
      "gross_revenue": number or null,
      "net_revenue": number or null,
      "discount_amount": number or null,
      "discount_percent": number or null,
      "profit": number or null,
      "rate_type": string or null,           // CPM, CPC, CPV, Flat, etc.
      "rate": number or null
    }
  ],
  "notes": string or null                     // Any additional context or clarifications
}

EXTRACTION RULES:
1. If a value is not present in the invoice, use null - DO NOT INVENT DATA.
2. If only one type of revenue is present, store it in gross_revenue and leave net_revenue null.
3. Discounts can be:
   - Explicit: directly stated as "discount" column
   - Implicit: difference between gross_revenue and net_revenue
   - Explain in 'notes' field if inferred
4. Profit calculation:
   - If stated directly, use that value
   - Otherwise: profit = net_revenue - cost (if cost is available)
   - If cannot be determined, leave as null
5. Views/Impressions:
   - Use the exact metric name from the invoice
   - Map "complete views", "video views", "conversions" to the closest field
   - Explain mapping in notes if ambiguous
6. Dates:
   - Convert to YYYY-MM-DD format when possible
   - Keep original format in notes if conversion is uncertain
7. Currency:
   - Extract currency code (USD, EUR, GBP, etc.) from invoice
8. Line Items:
   - Each row in a tabular invoice becomes one line_item
   - Assign sequential line_id starting from 1
9. Aggregation:
   - invoice_header totals should sum up from line_items when not explicitly stated
   - Flag any discrepancies in notes

BE CONSERVATIVE: Do not invent numbers. Use null for missing data.
"""

print("✅ Canonical Invoice Schema defined!")
print("   - Standard JSON structure for all invoice types")
print("   - Clear extraction rules for discounts, revenue, profit")
print("   - Handles PDF, Excel, CSV, and text formats")
CANONICAL_SCHEMA_DOC = """
You MUST output JSON ARRAY matching this structure:
 
[
  "invoice_header": {
    "invoice_number": string or null,
    "vendor_name": string or null,
    "campaign_name": string or null,
    "invoice_date": string or null,          // YYYY-MM-DD if possible
    "billing_start_date": string or null,    // YYYY-MM-DD if possible
    "billing_end_date": string or null,      // YYYY-MM-DD if possible
    "currency": string or null,           // YYYY-MM-DD if possible
      "impressions": number or null,
      "views": number or null,               // complete views, video views, completed clicks, clicks conversions, etc. choose the closest
      "gross_revenue": number or null,
      "net_revenue": number or null,
      "discount_amount": number or null,
      "discount_percent": number or null,
      "profit": number or null,
      "line_items": [
        {
          "line_id": integer,
          "campaign_name": string or null,
          "placement": string or null,
          "start_date": string or null,          // YYYY-MM-DD if possible
          "end_date": string or null,            // YYYY-MM-DD if possible
          "planned impressions": number or null,
          "billed impressions": number or null,
          "views": number or null,               // complete views, video views, completed clicks, clicks conversions, etc. choose the closest
          "gross_revenue": number or null,
          "net_revenue": number or null,
          "discount_amount": number or null,
          "discount_percent": number or null,
          "profit": number or null,
          "rate_type": string or null,           // CPM, CPC, CPV, Flat, etc.
          "rate": number or null,
        }
      ]
    },
  ]
 
RULES:
- If a value is not present in the invoice, use null.
- If only one type of revenue is present, store it in gross_revenue and leave net_revenue null (or vice versa if clearly net).
- Discounts can be explicit (discount column) or implicit (difference between gross and net) — explain in notes if inferred.
- Profit = revenue - cost, if not directly provided.
- Be conservative: do NOT invent numbers if they are not in the invoice.
"""

✅ Canonical Invoice Schema defined!
   - Standard JSON structure for all invoice types
   - Clear extraction rules for discounts, revenue, profit
   - Handles PDF, Excel, CSV, and text formats


In [150]:
# ============================================
# ENHANCED INVOICE CONTEXT BUILDER
# ============================================

import json
from typing import Optional, Dict, Any

def build_invoice_context(
    file_path: str,
    max_rows: int = 30
) -> Dict[str, Any]:
    """
    Build a unified context object from PDF/Excel/CSV for the agent.
    Returns structured context with type detection and preview.
    
    Args:
        file_path: Path to the invoice file
        max_rows: Maximum rows to include in preview (for tables)
    
    Returns:
        Dictionary with invoice context including file type and content
    """
    file_path_obj = Path(file_path)
    
    if not file_path_obj.exists():
        return {"error": f"File not found: {file_path}"}
    
    suffix = file_path_obj.suffix.lower()
    
    context = {
        "file_path": str(file_path),
        "file_name": file_path_obj.name,
        "input_type": None,
        "content": None
    }
    
    # PDF files
    if suffix == '.pdf':
        context["input_type"] = "pdf"
        # Read PDF text
        pdf_text = read_pdf_content(str(file_path))
        context["raw_text"] = pdf_text
        context["preview"] = pdf_text[:2000] + "..." if len(pdf_text) > 2000 else pdf_text
    
    # Excel/CSV files
    elif suffix in ['.xlsx', '.xls', '.csv']:
        context["input_type"] = "excel" if suffix in ['.xlsx', '.xls'] else "csv"
        
        # Read as DataFrame
        if suffix in ['.xlsx', '.xls']:
            df = pd.read_excel(file_path)
        else:
            df = pd.read_csv(file_path)
        
        # Clean column names
        df.columns = [
            str(c).strip().lower().replace(" ", "_").replace("-", "_")
            for c in df.columns
        ]
        
        # Create table preview
        preview_df = df.head(max_rows)
        
        context["table_preview"] = {
            "total_rows": len(df),
            "columns": list(df.columns),
            "column_types": {col: str(dtype) for col, dtype in df.dtypes.items()},
            "preview_rows": preview_df.to_dict(orient="records"),
            "preview_text": preview_df.to_string(index=False)
        }
    
    # Text files
    elif suffix == '.txt':
        context["input_type"] = "text"
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()
        context["raw_text"] = text
        context["preview"] = text[:2000] + "..." if len(text) > 2000 else text
    
    else:
        context["error"] = f"Unsupported file type: {suffix}"
    
    return context


def format_context_for_agent(context: Dict[str, Any]) -> str:
    """
    Format the invoice context into a readable string for the agent.
    """
    if "error" in context:
        return f"ERROR: {context['error']}"
    
    output = []
    output.append(f"FILE: {context['file_name']}")
    output.append(f"TYPE: {context['input_type'].upper()}")
    output.append("=" * 70)
    
    if context["input_type"] in ["pdf", "text"]:
        output.append("\nRAW TEXT CONTENT:")
        output.append(context.get("raw_text", ""))
    
    elif context["input_type"] in ["excel", "csv"]:
        table_preview = context.get("table_preview", {})
        output.append(f"\nTABLE STRUCTURE:")
        output.append(f"Total Rows: {table_preview.get('total_rows', 0)}")
        output.append(f"Columns: {', '.join(table_preview.get('columns', []))}")
        output.append(f"\nDATA PREVIEW (first {len(table_preview.get('preview_rows', []))} rows):")
        output.append(table_preview.get("preview_text", ""))
    
    return "\n".join(output)


print("✅ Invoice context builder functions created!")
print("   • build_invoice_context() - Unified file type handler")
print("   • format_context_for_agent() - Format for LLM consumption")


✅ Invoice context builder functions created!
   • build_invoice_context() - Unified file type handler
   • format_context_for_agent() - Format for LLM consumption


In [151]:
# ============================================
# ENHANCED INVOICE EXTRACTION TASK
# ============================================

def create_invoice_extraction_task(file_path: str, max_rows: int = 30) -> Task:
    """
    Create a task for extracting structured invoice data using the canonical schema.
    
    Args:
        file_path: Path to the invoice file (PDF, Excel, CSV, or text)
        max_rows: Maximum rows to include from tabular data
    
    Returns:
        Task object configured for invoice feature extraction
    """
    # Build context from file
    context = build_invoice_context(file_path, max_rows=max_rows)
    context_str = format_context_for_agent(context)
    
    description = f"""
You are analyzing a media/advertising invoice to extract key financial and delivery features.

**YOUR TASK:**
1. Understand the invoice header (vendor, invoice number, campaign, dates, currency)
2. Identify all line items (placements, quantities, prices, metrics)
3. Extract key features:
   - Discounts (amount and/or percent - explicit or implicit)
   - Impressions (planned vs. billed)
   - Views (complete views, video views, or nearest equivalent)
   - Clicks
   - Revenues (gross and net)
   - Costs
   - Profits
4. Map everything into the canonical JSON schema

**CANONICAL SCHEMA:**
{CANONICAL_SCHEMA_DOC}

**INVOICE DATA:**
{context_str}

**IMPORTANT INSTRUCTIONS:**
- Carefully scan all available fields and column names
- Look for columns like: impressions, views, complete_views, revenue, cost, profit, discount, etc.
- If only some metrics exist, fill those and leave the rest as null
- For Excel/CSV: each data row becomes one line_item with sequential line_id
- For PDF: extract line items from tables or structured text
- If discounts or profit need to be inferred (e.g., revenue - cost), compute it and explain in 'notes'
- Dates should be in YYYY-MM-DD format when possible
- Return ONLY the JSON object - no markdown, no explanations outside of JSON
- DO NOT INVENT DATA - use null for missing values

**OUTPUT:**
Return a single valid JSON object following the canonical schema exactly.
""".strip()
    
    task = Task(
        description=description,
        agent=file_reader_agent,
        expected_output="""A single valid JSON object with:
        - invoice_header: all header-level fields
        - line_items: array of line item objects
        - notes: any clarifications or assumptions made
        
        The JSON must be valid and parseable."""
    )
    
    return task


print("✅ Enhanced invoice extraction task creator defined!")
print("   Usage: task = create_invoice_extraction_task('invoice.xlsx')")


✅ Enhanced invoice extraction task creator defined!
   Usage: task = create_invoice_extraction_task('invoice.xlsx')


In [152]:
# ============================================
# MAIN INVOICE FEATURE EXTRACTION FUNCTION
# ============================================

def extract_invoice_features(file_path: str, max_rows: int = 30) -> Dict[str, Any]:
    """
    Extract structured invoice features using AI agent with canonical schema.
    
    Args:
        file_path: Path to invoice file (PDF, Excel, CSV, or text)
        max_rows: Maximum rows to process from tabular files
    
    Returns:
        Dictionary with extracted invoice data in canonical format
    """
    print(f"\n{'='*70}")
    print(f"📄 Extracting Invoice Features: {Path(file_path).name}")
    print(f"{'='*70}\n")
    
    # Create extraction task
    task = create_invoice_extraction_task(file_path, max_rows=max_rows)
    
    # Create crew
    crew = Crew(
        agents=[file_reader_agent],
        tasks=[task],
        process=Process.sequential,
        verbose=True
    )
    
    # Execute
    raw_result = crew.kickoff()
    result_str = str(raw_result).strip()
    
    print(f"\n{'='*70}")
    print(f"✅ Extraction Complete!")
    print(f"{'='*70}\n")
    
    # Try to parse JSON
    try:
        # First attempt: direct parse
        parsed = json.loads(result_str)
    except json.JSONDecodeError:
        # Fallback: find JSON object in the response
        print("⚠️  Direct JSON parse failed, attempting to extract JSON...")
        start = result_str.find("{")
        end = result_str.rfind("}")
        
        if start != -1 and end != -1 and start < end:
            json_str = result_str[start : end + 1]
            try:
                parsed = json.loads(json_str)
                print("✅ Successfully extracted JSON from response")
            except json.JSONDecodeError as e:
                print(f"❌ JSON parsing failed: {e}")
                return {
                    "error": "Failed to parse JSON response",
                    "raw_response": result_str[:500]
                }
        else:
            return {
                "error": "No JSON object found in response",
                "raw_response": result_str[:500]
            }
    
    return parsed


print("✅ Main invoice feature extraction function created!")
print("   Usage: result = extract_invoice_features('invoice.xlsx')")
print("   Returns: Structured JSON with invoice_header, line_items, and notes")


✅ Main invoice feature extraction function created!
   Usage: result = extract_invoice_features('invoice.xlsx')
   Returns: Structured JSON with invoice_header, line_items, and notes


In [153]:
# ============================================
# RESULT VALIDATION & EXPORT HELPERS
# ============================================

def validate_invoice_result(result: Dict[str, Any]) -> Dict[str, Any]:
    """
    Validate the extracted invoice data against the canonical schema.
    
    Returns:
        Dictionary with validation status and any issues found
    """
    validation = {
        "valid": True,
        "errors": [],
        "warnings": []
    }
    
    # Check for top-level structure
    if "invoice_header" not in result:
        validation["valid"] = False
        validation["errors"].append("Missing 'invoice_header' field")
    
    if "line_items" not in result:
        validation["valid"] = False
        validation["errors"].append("Missing 'line_items' field")
    elif not isinstance(result["line_items"], list):
        validation["valid"] = False
        validation["errors"].append("'line_items' must be an array")
    
    # Check invoice_header fields
    if "invoice_header" in result:
        header = result["invoice_header"]
        if not header.get("invoice_number") and not header.get("vendor_name"):
            validation["warnings"].append("Missing both invoice_number and vendor_name in header")
    
    # Check line_items structure
    if "line_items" in result and isinstance(result["line_items"], list):
        for idx, item in enumerate(result["line_items"]):
            if "line_id" not in item:
                validation["warnings"].append(f"Line item {idx} missing 'line_id'")
    
    return validation


def save_invoice_json(result: Dict[str, Any], output_path: str = None) -> str:
    """
    Save the extracted invoice data to a JSON file.
    
    Args:
        result: Extracted invoice data
        output_path: Path to save JSON (optional, auto-generated if not provided)
    
    Returns:
        Path to saved file
    """
    if output_path is None:
        # Auto-generate filename
        timestamp = pd.Timestamp.now().strftime("%Y%m%d_%H%M%S")
        output_path = f"data/extracted_invoice_{timestamp}.json"
    
    # Ensure directory exists
    Path(output_path).parent.mkdir(parents=True, exist_ok=True)
    
    # Save JSON
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(result, f, indent=2, ensure_ascii=False)
    
    return output_path


def invoice_to_dataframe(result: Dict[str, Any]) -> pd.DataFrame:
    """
    Convert extracted invoice line items to a pandas DataFrame.
    
    Args:
        result: Extracted invoice data with line_items
    
    Returns:
        DataFrame with all line items
    """
    if "line_items" not in result or not result["line_items"]:
        return pd.DataFrame()
    
    df = pd.DataFrame(result["line_items"])
    
    # Add header information to each row
    if "invoice_header" in result:
        header = result["invoice_header"]
        df["invoice_number"] = header.get("invoice_number")
        df["vendor_name"] = header.get("vendor_name")
        df["invoice_date"] = header.get("invoice_date")
    
    return df


print("✅ Validation and export helper functions created!")
print("   • validate_invoice_result() - Check schema compliance")
print("   • save_invoice_json() - Save to JSON file")
print("   • invoice_to_dataframe() - Convert to pandas DataFrame")


✅ Validation and export helper functions created!
   • validate_invoice_result() - Check schema compliance
   • save_invoice_json() - Save to JSON file
   • invoice_to_dataframe() - Convert to pandas DataFrame


## 🚀 Example: Extract Invoice Features with Canonical Schema

Now let's test the enhanced invoice extraction with the improved prompts and schema.


In [ ]:
# ============================================
# EXAMPLE: EXTRACT FROM SAMPLE INVOICE
# ============================================

# Extract features from the sample invoice we created earlier
result = extract_invoice_features('data/sample_media_invoice.xlsx', max_rows=50)

# Display the results
print("\n" + "="*70)
print("📊 EXTRACTED INVOICE DATA")
print("="*70)
print(json.dumps(result, indent=2))

# Validate the result
print("\n" + "="*70)
print("✅ VALIDATION")
print("="*70)
validation = validate_invoice_result(result)
print(f"Valid: {validation['valid']}")
if validation['errors']:
    print(f"Errors: {validation['errors']}")
if validation['warnings']:
    print(f"Warnings: {validation['warnings']}")

# Save to JSON
output_file = save_invoice_json(result, "data/extracted_sample_invoice.json")
print(f"\n💾 Saved to: {output_file}")

# Convert to DataFrame
df = invoice_to_dataframe(result)
print(f"\n📋 DataFrame Preview:")
print(df.head())


## 📄 Process Your Real Invoice File

Use this cell to process your actual invoice file with the enhanced extraction.


In [ ]:
# ============================================
# PROCESS YOUR INVOICE FILE
# ============================================

# Step 1: Check the file structure first (optional but recommended)
print("📋 File Structure Analysis:")
summary = get_excel_summary('data/All Raw data.xlsx', concise=True)
print(summary)

print("\n" + "="*70 + "\n")

# Step 2: Extract invoice features with canonical schema
# Note: This will process the first sheet with up to 50 rows
# Adjust max_rows as needed, but keep it reasonable to avoid token limits

invoice_data = extract_invoice_features(
    'data/All Raw data.xlsx',
    max_rows=50  # Limit rows to prevent token overflow
)

# Step 3: Display results
print("\n" + "="*70)
print("📊 EXTRACTION RESULTS")
print("="*70)

if "error" in invoice_data:
    print(f"❌ Error: {invoice_data['error']}")
else:
    print("\n📋 Invoice Header:")
    print(json.dumps(invoice_data.get("invoice_header", {}), indent=2))
    
    print(f"\n📦 Line Items: {len(invoice_data.get('line_items', []))}")
    if invoice_data.get('line_items'):
        print("\nFirst 3 line items:")
        for item in invoice_data['line_items'][:3]:
            print(f"  Line {item.get('line_id')}: {item.get('campaign_name', 'N/A')}")
            print(f"    Revenue: {item.get('gross_revenue', 'N/A')}, Impressions: {item.get('billed_impressions', 'N/A')}")
    
    if invoice_data.get('notes'):
        print(f"\n📝 Notes: {invoice_data['notes']}")
    
    # Step 4: Validate
    validation = validate_invoice_result(invoice_data)
    print(f"\n✅ Validation: {'PASS' if validation['valid'] else 'FAIL'}")
    if validation['warnings']:
        print(f"⚠️  Warnings: {', '.join(validation['warnings'])}")
    
    # Step 5: Save results
    output_file = save_invoice_json(invoice_data, "data/extracted_invoice_features.json")
    print(f"\n💾 Saved to: {output_file}")
    
    # Step 6: Convert to DataFrame for analysis
    df = invoice_to_dataframe(invoice_data)
    if not df.empty:
        print(f"\n📊 DataFrame created with {len(df)} rows")
        print("\nSample data:")
        print(df[['line_id', 'campaign_name', 'gross_revenue', 'billed_impressions']].head())


## 🚀 Step 6: Create and Run Crew

Assemble the crew and execute the file reading task.

In [121]:
# ============================================
# CREATE AND RUN FILE READER CREW
# ============================================

def process_file(file_path: str, extraction_focus: str = "all data"):
    """
    Process a file using the File Reader Agent and Crew.
    
    Args:
        file_path: Path to the file to process
        extraction_focus: Specific data to focus on
        
    Returns:
        Extracted data and analysis
    """
    print(f"\n{'='*70}")
    print(f"🚀 Processing File: {Path(file_path).name}")
    print(f"{'='*70}\n")
    
    # Create task for this file
    task = create_file_reading_task(file_path, extraction_focus)
    
    # Create crew
    crew = Crew(
        agents=[file_reader_agent],
        tasks=[task],
        process=Process.sequential,
        verbose=True
    )
    
    # Execute
    result = crew.kickoff()
    
    print(f"\n{'='*70}")
    print(f"✅ File Processing Complete!")
    print(f"{'='*70}\n")
    
    return result

print("✅ File processing function created!")
print("   Usage: result = process_file('path/to/invoice.pdf')")

✅ File processing function created!
   Usage: result = process_file('path/to/invoice.pdf')
